### Importações das bibliotecas

In [1]:
# Importação para ignorar os avisos das bibliotecas (famoso future_warning)
import warnings
warnings.filterwarnings('ignore')
# Análise e manipulação
import pandas as pd
import numpy as np
import sys
# Transformações e treinamentos
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Modelos
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor
# Salvar os modelos
import joblib
import pickle

In [2]:
airbnb = pd.read_csv("datasets/listings.csv")

df = airbnb[['neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'accommodates', 'bathrooms', 'bathrooms_text', 
             'bedrooms', 'beds', 'minimum_nights', 'availability_30', 'availability_60', 'availability_90','availability_365', 
             'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
             'review_scores_communication', 'review_scores_location', 'review_scores_value', 
             'price']].copy()

### Tratamento dos dados ausentes (NaN)

In [3]:
# Apagando a coluna banheiros
df.drop(['bathrooms'], axis = 1, inplace = True)

In [4]:
valores = {'bathrooms_text': '1 bath', 
           'bedrooms': 1.0, 
           'beds': 1.0,
           'review_scores_rating': 0,
           'review_scores_accuracy': 0,
           'review_scores_cleanliness': 0,
           'review_scores_checkin': 0,
           'review_scores_communication': 0,
           'review_scores_location': 0,
           'review_scores_value': 0}

df = df.fillna(value = valores)

### Verificação e tratamento de dados duplicados

In [5]:
#Verificando a quantidade de dados duplicados
print("Valores duplicados antes da transformação: \n\n{}\n".format(df.duplicated().sum()))

df.drop_duplicates(inplace = True)

print("Valores duplicados após a transformação: \n\n{}\n".format(df.duplicated().sum()))

Valores duplicados antes da transformação: 

7

Valores duplicados após a transformação: 

0



### Verificação e tratamento do tipo das colunas

In [6]:
# Alteração para o tipo int da coluna bedrooms
df['bedrooms'] = df['bedrooms'].astype(int)
# Alteração para o tipo int da coluna beds
df['beds'] = df['beds'].astype(int)

In [7]:
# Filtrando a nossa coluna bathrooms_text
df['bathrooms_text'] = df['bathrooms_text'].str.split(' ').str[0]

df.loc[df['bathrooms_text'] == "Shared", 'bathrooms_text'] = 1
df.loc[df['bathrooms_text'] == "Half-bath", 'bathrooms_text'] = 1
df.loc[df['bathrooms_text'] == "Private", 'bathrooms_text'] = 1
# Alteração para o tipo int da coluna bathrooms_text
df['bathrooms_text'] = df['bathrooms_text'].astype(float)
df['bathrooms_text'] = df['bathrooms_text'].astype(int)

In [8]:
# Alteração para o tipo int da coluna review_scores_rating
df['review_scores_rating'] = df['review_scores_rating'].astype(int)
# Alteração para o tipo int da coluna review_scores_accuracy
df['review_scores_accuracy'] = df['review_scores_accuracy'].astype(int)
# Alteração para o tipo int da coluna review_scores_cleanliness
df['review_scores_cleanliness'] = df['review_scores_cleanliness'].astype(int)
# Alteração para o tipo int da coluna review_scores_checkin
df['review_scores_checkin'] = df['review_scores_checkin'].astype(int)
# Alteração para o tipo int da coluna review_scores_communication
df['review_scores_communication'] = df['review_scores_communication'].astype(int)
# Alteração para o tipo int da coluna review_scores_location
df['review_scores_location'] = df['review_scores_location'].astype(int)
# Alteração para o tipo int da coluna review_scores_value
df['review_scores_value'] = df['review_scores_value'].astype(int)

In [9]:
# Antes de alterarmos, devemos retirar '$' e ','
df['price'] = df.price.str.replace('$', '')
df['price'] = df.price.str.replace(',', '')
# Alterando para tipo float64
df['price'] = pd.to_numeric(df.price)

In [10]:
display(df['price'].sample(3))

df['price'].describe()

22230    218.0
2376     400.0
22037    270.0
Name: price, dtype: float64

count     26608.000000
mean        742.717529
std        5369.569077
min           0.000000
25%         157.000000
50%         280.000000
75%         550.000000
max      625216.000000
Name: price, dtype: float64

### Verificação de dados inconsistentes

In [11]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

#### Filtrando valores númericos

In [12]:
df.describe()

,latitude,longitude,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,availability_30,availability_60,availability_90,availability_365,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
count,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000,26608.000000
mean,-22.965842,-43.248541,4.160891,1.575466,1.661493,2.533561,4.725346,19.250263,41.232223,64.250338,219.482186,57.264770,5.830126,5.683366,5.929683,5.906118,5.905367,5.625601,742.717529
std,0.034972,0.096293,2.494420,1.005238,1.034164,2.122884,19.105020,12.452754,23.788491,34.633822,141.513264,46.837873,4.767351,4.675823,4.827624,4.813450,4.806387,4.617485,5369.569077
min,-23.072920,-43.704790,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-22.984570,-43.304100,2.000000,1.000000,1.000000,1.000000,1.000000,3.000000,23.000000,43.000000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,157.000000
50%,-22.971705,-43.196220,4.000000,1.000000,1.000000,2.000000,2.000000,27.000000,56.500000,86.000000,255.000000,90.000000,9.000000,9.000000,10.000000,10.000000,10.000000,9.000000,280.000000
75%,-22.951607,-43.186300,5.000000,2.000000,2.000000,3.000000,4.000000,30.000000,59.000000,89.000000,363.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,550.000000
max,-22.749820,-43.104860,16.000000,20.000000,30.000000,50.000000,1000.000000,30.000000,60.000000,90.000000,365.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,625216.000000


###### Coluna ``price``

In [13]:
# Filtrando...
df = df[df.price > 0]
df = remove_outlier(df, 'price')

display(df['price'].describe())

count    23831.000000
mean       329.085687
std        245.506407
min         33.000000
25%        150.000000
50%        250.000000
75%        443.000000
max       1139.000000
Name: price, dtype: float64

###### Coluna ``accommodates``

In [14]:
df['accommodates'] = pd.cut(df['accommodates'], 
                    bins = [0, 1, 2, 3, 4, 5, 6, sys.maxsize], 
                    labels = ['1', '2', '3', '4', '5', '6', '7+'])

df['accommodates'] = df['accommodates'].astype(object)

###### Coluna ``bathrooms_text``

In [15]:
# Filtrando...
df = df[(df.bathrooms_text > 0)]

df['bathrooms_text'] = pd.cut(df['bathrooms_text'], 
                    bins = [0, 1, 2, 3, sys.maxsize], 
                    labels = ['1', '2', '3', '4+'])

df['bathrooms_text'] = df['bathrooms_text'].astype(object)

###### Coluna ``bedrooms``

In [16]:
df['bedrooms'] = pd.cut(df['bedrooms'], 
                    bins = [0, 1, 2, 3, sys.maxsize], 
                    labels = ['1', '2', '3', '4+'])

df['bedrooms'] = df['bedrooms'].astype(object)

###### Coluna ``beds``

In [17]:
# Filtrando...
df = df[(df.beds > 0)]

df['beds'] = pd.cut(df['beds'], 
                    bins = [0, 1, 2, 3, sys.maxsize], 
                    labels = ['1', '2', '3', '4+'])

df['beds'] = df['beds'].astype(object)

###### Coluna ``minimum_nights``

In [18]:
# Filtrando...
df = df[df.minimum_nights <= 360]
# df = remove_outlier(df, 'minimum_nights')

df['minimum_nights'] = pd.cut(df['minimum_nights'], 
                    bins = [0, 1, 2, 3, 4, sys.maxsize], 
                    labels = ['1', '2', '3', '4', '5+'])

# Trocando o tipo da coluna para 'object'
df['minimum_nights'] = df['minimum_nights'].astype(object)

###### Coluna ``review_scores_rating``

In [19]:
df['review_scores_rating'] = pd.cut(df['review_scores_rating'], 
                    bins = [-sys.maxsize - 1, 0, 75, 90, 99, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Bom', 'Otimo', 'Perfeito'])

df['review_scores_rating'] = df['review_scores_rating'].astype(object)

###### Todas as outras colunas ``review_scores``

In [20]:
df['review_scores_accuracy'] = pd.cut(df['review_scores_accuracy'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_accuracy'] = df['review_scores_accuracy'].astype(object)

In [21]:
df['review_scores_cleanliness'] = pd.cut(df['review_scores_cleanliness'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_cleanliness'] = df['review_scores_cleanliness'].astype(object)

In [22]:
df['review_scores_checkin'] = pd.cut(df['review_scores_checkin'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_checkin'] = df['review_scores_checkin'].astype(object)

In [23]:
df['review_scores_communication'] = pd.cut(df['review_scores_communication'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_communication'] = df['review_scores_communication'].astype(object)

In [24]:
df['review_scores_location'] = pd.cut(df['review_scores_location'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_location'] = df['review_scores_location'].astype(object)

In [25]:
df['review_scores_value'] = pd.cut(df['review_scores_value'], 
                    bins = [-sys.maxsize - 1, 0, 5, 7, 9, sys.maxsize], 
                    labels = ['Sem Resposta', 'Razoavel', 'Regular', 'Bom', 'Otimo'])

df['review_scores_value'] = df['review_scores_value'].astype(object)

#### Filtrando valores categóricos

In [26]:
df.describe(include = 'O')

,neighbourhood_cleansed,property_type,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
count,23165,23165,23165,23165,23165,23165,23165,23165,23165,23165,23165,23165,23165,23165
unique,149,83,7,4,4,4,5,5,5,5,5,5,5,5
top,Copacabana,Entire apartment,2,1,1,1,1,Sem Resposta,Otimo,Otimo,Otimo,Otimo,Otimo,Sem Resposta
freq,6948,13219,6410,15500,14649,8211,6167,8208,11716,9537,13271,12891,12685,8217


In [27]:
df['neighbourhood_region'] = df['neighbourhood_cleansed'].copy()

df.neighbourhood_region.replace({
    'Copacabana':'Zona Sul',
    'Ipanema':'Zona Sul',
    'Barra da Tijuca':'Zona Oeste',
    'Botafogo':'Zona Sul',
    'Jacarepaguá':'Zona Oeste',
    'Leblon':'Zona Sul',
    'Recreio dos Bandeirantes':'Zona Oeste',
    'Santa Teresa':'Zona Sul',
    'Centro':'Zona Central',
    'Flamengo':'Zona Sul',
    'Leme':'Zona Sul',
    'Tijuca':'Zona Norte',
    'Laranjeiras':'Zona Sul',
    'Catete':'Zona Sul',
    'Camorim':'Zona Oeste',
    'Lagoa':'Zona Sul',
    'Glória':'Zona Sul',
    'Jardim Botânico':'Zona Sul',
    'Vila Isabel':'Zona Norte',
    'São Conrado':'Zona Sul', 
    'Humaitá':'Zona Sul',
    'Gávea':'Zona Sul',
    'Maracanã':'Zona Norte',
    'Vidigal':'Zona Sul',
    'Freguesia (Jacarepaguá)':'Zona Oeste',
    'Taquara':'Zona Oeste',
    'Urca':'Zona Sul',
    'Vargem Pequena':'Zona Oeste',
    'Itanhangá':'Zona Oeste',
    'Rio Comprido':'Zona Central', 
    'Vargem Grande':'Zona Oeste',
    'Campo Grande':'Zona Oeste',
    'Cosme Velho':'Zona Sul',
    'São Cristóvão':'Zona Central',
    'Estácio':'Zona Central',
    'Guaratiba':'Zona Oeste',
    'Barra de Guaratiba':'Zona Oeste',
    'Jardim Guanabara':'Zona Norte',
    'Praça da Bandeira':'Zona Norte',
    'Curicica':'Zona Oeste',
    'Grajaú':'Zona Norte',
    'Pechincha':'Zona Oeste',
    'Andaraí':'Zona Norte',
    'Anil':'Zona Oeste',
    'Paquetá':'Zona Central',
    'Engenho Novo':'Zona Norte',
    'Irajá':'Zona Norte',
    'Alto da Boa Vista':'Zona Norte',
    'Praça da Bandeira':'Zona Norte',
    'Saúde':'Zona Central',
    'Todos os Santos':'Zona Norte',
    'Brás de Pina':'Zona Norte',
    'Méier':'Zona Norte',
    'Cachambi':'Zona Norte',
    'Portuguesa':'Zona Norte',
    'Joá':'Zona Oeste',
    'Praça Seca':'Zona Oeste',
    'Bonsucesso':'Zona Norte',
    'Cidade Nova':'Zona Central',
    'Rocinha':'Zona Sul',
    'Rocha':'Zona Norte',
    'Penha':'Zona Norte',
    'Santa Cruz':'Zona Oeste',
    'São Francisco Xavier':'Zona Norte',
    'Bangu':'Zona Oeste',
    'Gardênia Azul':'Zona Oeste',
    'Encantado':'Zona Norte',
    'Vila Valqueire':'Zona Oeste',
    'Riachuelo':'Zona Norte',
    'Tanque':'Zona Oeste',
    'Cidade de Deus':'Zona Oeste',
    'Lins de Vasconcelos':'Zona Norte',
    'Gamboa':'Zona Central',
    'Del Castilho':'Zona Norte',
    'Maria da Graça':'Zona Norte',
    'Piedade':'Zona Norte',
    'Pilares':'Zona Norte',
    'Higienópolis':'Zona Norte',
    'Santo Cristo':'Zona Central',
    'Tauá':'Zona Norte',
    'Vila da Penha':'Zona Norte',
    'Marechal Hermes':'Zona Norte',
    'Santíssimo':'Zona Oeste',
    'Campinho':'Zona Norte',
    'Vasco da Gama':'Zona Norte',
    'Penha Circular':'Zona Norte',
    'Pilares':'Zona Norte',
    'Bento Ribeiro':'Zona Norte',
    'Ramos':'Zona Norte',
    'Pedra de Guaratiba':'Zona Oeste',
    'Jardim Sulacap':'Zona Oeste',
    'Benfica':'Zona Central',
    'Padre Miguel':'Zona Oeste',
    'Catumbi':'Zona Central',
    'Cosmos':'Zona Oeste',
    'Sampaio':'Zona Norte',
    'Ricardo de Albuquerque':'Zona Norte',
    'Mangueira':'Zona Central',
    'Cocotá':'Zona Norte',
    'Parada de Lucas':'Zona Norte',   
    'Olaria':'Zona Norte',
    'Cocotá':'Zona Norte',
    'Senador Camará':'Zona Oeste',
    'Sepetiba':'Zona Oeste',
    'Coelho Neto':'Zona Norte',
    'Senador Vasconcelos':'Zona Oeste',
    'Vicente de Carvalho':'Zona Norte',
    'Pitangueiras':'Zona Norte',
    'Cascadura':'Zona Norte',
    'Tomás Coelho':'Zona Norte',
    'Inhaúma':'Zona Norte',
    'Quintino Bocaiúva':'Zona Norte',
    'Inhoaíba':'Zona Oeste',
    'Pavuna':'Zona Norte',
    'Madureira':'Zona Norte',
    'Jardim Carioca':'Zona Norte',
    'Praia da Bandeira':'Zona Norte',
    'Cidade Universitária':'Zona Norte',
    'Barros Filho':'Zona Norte',
    'Realengo':'Zona Oeste',
    'Manguinhos':'Zona Norte',
    'Ribeira':'Zona Norte',
    'Moneró':'Zona Norte',
    'Cordovil':'Zona Norte',
    'Complexo do Alemão':'Zona Norte',
    'Maré':'Zona Norte',
    'Acari':'Zona Norte',
    'Freguesia (Ilha)':'Zona Norte',
    'Cacuia':'Zona Norte',
    'Vigário Geral':'Zona Norte',
    'Água Santa':'Zona Norte',
    'Magalhães Bastos':'Zona Oeste',
    'Cavalcanti':'Zona Norte',
    'Gericinó':'Zona Oeste',
    'Honório Gurgel':'Zona Norte',
    'Zumbi':'Zona Norte',
    'Osvaldo Cruz':'Zona Norte',
    'Vaz Lobo':'Zona Norte',
    'Paciência':'Zona Oeste',
    'Grumari':'Zona Oeste',
    'Vila Militar':'Zona Oeste',
    'Parque Anchieta':'Zona Norte',
    'Deodoro':'Zona Oeste',
    'Guadalupe':'Zona Norte',
    'Rocha Miranda':'Zona Norte',
    'Engenho de Dentro':'Zona Norte',
    'Engenho da Rainha':'Zona Norte',
    'Abolição':'Zona Norte',
    'Jacaré':'Zona Norte',
    'Galeão':'Zona Norte',
    'Bancários':'Zona Norte',
    'Vila Kosmos':'Zona Norte'}, inplace = True)

###### Coluna ``property_type``

In [28]:
df.property_type.replace({
    'Entire apartment':'Apartment',
    'Entire condominium':'Apartment',
    'Entire floor':'Apartament',
    'Entire home/apt':'Apartment',
    'Entire serviced apartment':'Apartment',
    'Entire loft':'Apartment',
    'Private room in apartment':'Room/House',
    'Private room in house':'Room/House',
    'Private room in condominium':'Room/House',
    'Shared room in apartment':'Room/House',
    'Shared room in house':'Room/House',
    'Private room in bed and breakfast':'Room/House',
    'Private room in guest suite':'Room/House',
    'Room in hotel':'Room/House',
    'Room in aparthotel':'Room/House',
    'Entire guest suite':'Room/House',
    'Private room in serviced apartment':'Room/House',
    'Private room in guesthouse':'Room/House',
    'Shared room in hostel':'Room/House',
    'Private room in villa':'Room/House',
    'Room in boutique hotel':'Room/House',
    'Private room in loft':'Room/House',
    'Private room in hostel':'Room/House',
    'Private room in townhouse':'Room/House',
    'Room in serviced apartment':'Room/House',
    'Shared room in condominium':'Room/House',
    'Private room in earth house':'Room/House',
    'Shared room in bed and breakfast':'Room/House',
    'Room in hostel':'Room/House',
    'Shared room in guesthouse':'Room/House',
    'Room in bed and breakfast':'Room/House',
    'Private room in casa particular':'Room/House',
    'Shared room in guest suite':'Room/House',
    'Private room':'Room/House',
    'Shared room in loft':'Room/House',
    'Private room in tiny house':'Room/House',
    'Private room in cottage':'Room/House',
    'Private room in chalet':'Room/House',
    'Shared room in villa':'Room/House',
    'Private room in nature lodge':'Room/House',
    'Shared room in tiny house':'Room/House',
    'Shared room in casa particular':'Room/House',
    'Private room in island':'Room/House',
    'Private room in bungalow':'Room/House',
    'Shared room in earth house':'Room/House',
    'Shared room in boutique hotel':'Room/House',
    'Room in nature lodge':'Room/House',
    'Private room in floor':'Room/House',
    'Entire dorm':'Room/House',
    'Private room in barn':'Room/House',
    'Private room in castle':'Room/House',
    'Shared room':'Room/House',
    'Shared room in townhouse':'Room/House',
    'Private room in treehouse':'Room/House',
    'Shared room in farm stay':'Room/House',
    'Private room in boat':'Room/House',
    'Private room in farm stay':'Room/House',
    'Shared room in cottage':'Room/House',
    'Entire vacation home':'Room/House',
    'Shared room in nature lodge':'Room/House',
    'Shared room in bungalow':'Room/House',
    'Private room in tent':'Room/House',
    'Shared room in castle':'Room/House',
    'Private room in cabin':'Room/House',
    'Entire house':'Room/House',
    'Entire villa':'Room/House',
    'Entire guesthouse':'Room/House',
    'Tiny house':'Room/House',
    'Entire cottage':'Room/House',
    'Entire townhouse':'Room/House',
    'Entire place':'Room/House',
    'Entire chalet':'Room/House',
    'Earth house':'Room/House',
    'Casa particular':'Room/House',
    'Entire cabin':'Room/House',
    'Entire bungalow':'Room/House',
    'Houseboat':'Room/House',
    'Farm stay':'Room/House',
    'Treehouse':'Room/House',
    'Hut':'Room/House',
    'Entire vacation home':'Room/House',
    'Entire hostel':'Room/House',
    'Entire floor':'Room/House',
    'Entire bed and breakfast':'Room/House',
    'Island':'Room/House',
    'Yurt':'Room/House',
    'Camper/RV': 'Room/House',
    'Campsite': 'Room/House',
    'Boat': 'Room/House'
    }, inplace = True)

## Modelagem e Avaliação de Modelos de Classificação

### Seleção de Variaveis características e variaveis-alvo

In [29]:
# Definição das colunas que serão features
X_c = df[['neighbourhood_region', 'latitude', 'longitude','accommodates', 'bathrooms_text', 'bedrooms', 'beds', 
              'minimum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
              'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
              'review_scores_checkin', 'review_scores_communication',
              'review_scores_location', 'review_scores_value', 
              'price']].copy()

# Definição da variável-alvo
y_c = df['property_type'].copy()
# Transformação da nossa variável alvo para tipo númerico
y_c.replace({'Apartment': 0, 'Room/House': 1}, inplace = True)

### Pré-processamento dos dados

In [30]:
# Pré-processamento das features categóricas (OneHotEncoder)
categorical_features_ohe = [0, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17, 18]
categorical_transformer_ohe= Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [31]:
# Pré-processamento das features numéricas (StandardScaler)
numeric_features = [1, 2, 8, 9, 10, 11, 19]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [32]:
# Juntando o pré-processamento das features categóricas e númericas
preprocessor_classification = ColumnTransformer(
    transformers=[
        ('cat_ohe', categorical_transformer_ohe, categorical_features_ohe),
        ('num', numeric_transformer, numeric_features)])

### Modelagem dos modelos de Classificação

###### XGBoost (Only Classification)

In [33]:
# Criando o pré-processamento e o Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor_classification),
                           ('classification', XGBClassifier(n_estimators = 1500, max_depth = 30, learning_rate = 0.01, 
                                                            colsample_bytree = 0.4))])


# Utilizando o pipeline para treinar o modelo de XGBoost na base de treinamento
xgbc = pipeline.fit(X_c, (y_c))

## Modelagem e Avaliação de Modelos de Regressão

### Seleção de Variaveis características e variaveis-alvo

In [34]:
# Definição das colunas que serão features (lembrando que a coluna 'price' não está presente)
X_r = df[['neighbourhood_region', 'latitude', 'longitude', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 
          'minimum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
          'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
          'review_scores_checkin', 'review_scores_communication', 
          'review_scores_location', 'review_scores_value', 
          'property_type']].copy()

# Definição da variável-alvo
y_r = df['price'].copy()

### Pré-processamento dos dados

In [35]:
# Pré-processamento das features categóricas (OneHotEncoder)
categorical_features_ohe = [0, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17, 18, 19]
categorical_transformer_ohe= Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [36]:
# Pré-processamento das features numéricas (StandardScaler)
numeric_features = [1, 2, 8, 9, 10, 11]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [37]:
# Juntando o pré-processamento das features categóricas e númericas
preprocessor_regression = ColumnTransformer(
    transformers=[
        ('cat_ohe', categorical_transformer_ohe, categorical_features_ohe),
        ('num', numeric_transformer, numeric_features)])

### Modelagem e Avaliação dos modelos

###### LGBM (Only Regression)

In [38]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor_regression),
                           ('model_lgbm', LGBMRegressor(n_estimators = 70, num_leaves = 50, learning_rate = 0.1, 
                                                        boosting_type = 'gbdt',objective = 'regression', 
                                                        metric = 'mae'))])

lgbmr = pipeline.fit(X_r, y_r)

## Modelagem e Avaliação de Modelos de Classificação e Regressão

### Seleção de Variaveis características e variaveis-alvo

In [39]:
# Definição das colunas que serão features
X_cr = df[['neighbourhood_region', 'latitude', 'longitude', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 
           'minimum_nights', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
           'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
           'review_scores_checkin', 'review_scores_communication', 
           'review_scores_location', 'review_scores_value']].copy()

# Definição da variável-alvo
y_cr = df['property_type'].copy()
y_rc = df['price'].copy()

### Pré-processamento dos dados

In [40]:
# Pré-processamento das features categóricas (OneHotEncoder)
categorical_features_ohe = [0, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17, 18]
categorical_transformer_ohe= Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [41]:
numeric_features = [1, 2, 8, 9, 10, 11]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat_ohe', categorical_transformer_ohe, categorical_features_ohe),
        ('num', numeric_transformer, numeric_features)])

###### XGBoost (Classification & Regression)

In [43]:
# Criando o pré-processamento e o Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classification', XGBClassifier(n_estimators = 1500, max_depth = 30, learning_rate = 0.01,
                                                            colsample_bytree = 0.4))])


# Utilizando o pipeline para treinar o modelo de XGBoost na base de treinamento
xgbcr = pipeline.fit(X_cr, (y_cr))

###### LGBM (Classification & Regression)

In [44]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model_lgbm', LGBMRegressor(n_estimators = 70, num_leaves = 50, learning_rate = 0.1, 
                                                        boosting_type = 'gbdt',objective = 'regression', 
                                                        metric = 'mae'))])

lgbmcr = pipeline.fit(X_cr, y_rc)

### Salvando os modelos

In [45]:
# Salvando o modelo de XGBoost de Classificação & Regressão
joblib.dump(xgbc, '../models/XGboost_class_airbnb.pkl')
joblib.dump(xgbcr, '../models/XGboost_classreg_airbnb.pkl')
# Salvando o modelo de LGBM de Regressão & Classificação
joblib.dump(lgbmr, '../models/LGBM_reg_airbnb.pkl')
joblib.dump(lgbmcr, '../models/LGBM_classreg_airbnb.pkl')

['../models/LGBM_classreg_airbnb.pkl']